In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_KEY")
os.environ['LANGCHAIN_PROJECT'] = "earning_call_eval_v3"

In [2]:
import json
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from ragas import evaluate
from datasets import Dataset
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
import pandas as pd

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)
embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")

llm = LangchainLLMWrapper(langchain_llm=llm)
embeddings = LangchainEmbeddingsWrapper(embeddings)

c:\Users\HP\miniconda3\envs\langchain_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
import os

eval_dir = "eval_data_json/"
filenames = os.listdir(eval_dir)
filenames[:5]

['Adani Enterprises Ltd.json',
 'Adani Ports and Special Economic Zone Ltd.json',
 'Apollo Hospitals.json',
 'Asian Paints.json',
 'Axis Bank.json']

In [22]:
def create_mini_batches(data_dict, num_partitions=3):
    # Get the length of the questions list
    total_length = len(data_dict["question"])
    print(total_length)
    
    # Calculate the size of each partition
    partition_size = total_length // num_partitions
    
    # Create a list to store the mini-batches
    mini_batches = []
    
    # Iterate through the data in partitions
    for i in range(num_partitions):
        start_idx = i * partition_size
        # Ensure the last partition includes any remaining elements
        if i == num_partitions - 1:
            end_idx = total_length
        else:
            end_idx = (i + 1) * partition_size
        
        # Create a mini-batch dictionary
        mini_batch = {
            "question": data_dict["question"][start_idx:end_idx],
            "answer": data_dict["answer"][start_idx:end_idx],
            "contexts": data_dict["contexts"][start_idx:end_idx],
            "ground_truth": data_dict["ground_truth"][start_idx:end_idx]
        }
        
        # Append the mini-batch to the list
        print(start_idx, end_idx)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [25]:
import time

for filename in filenames[11:21]:
    i = 0
    with open(eval_dir + filename, 'r') as file:
        json_data = json.load(file)

    answers = []
    questions = []
    contexts = []
    ground_truths = []

    for item in json_data:
        answers.append(item['answer'])
        questions.append(item['question'])
        contexts.append(item['contexts'])
        ground_truths.append(item['ground_truth'])

    data_dict = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truth": ground_truths
    }
    mini_batches = create_mini_batches(data_dict, num_partitions=5)
    list_df = []
    # runing this loop over mini-batches since passing all data at once gives token_per_minute limit excedded error.
    for mini_batch in mini_batches:
        dataset = Dataset.from_dict(mini_batch)
        result = evaluate(dataset, llm=llm, embeddings=embeddings, metrics=[faithfulness, answer_relevancy, context_precision, context_recall])
        df = result.to_pandas()
        list_df.append(df)
        if (i+1) % 2 == 0:
            print(f"SLEEPING at {i+1}")
            time.sleep(60)
        i+=1
    
    concatenated_df = pd.concat(list_df, ignore_index=True)
    print(f"> {filename}, Shape: {concatenated_df.shape}")
    concatenated_df.to_csv(f"eval_out/{filename.replace('.json', '.csv')}", index=False)

10
0 2
2 4
4 6
6 8
8 10


Evaluating:   0%|          | 0/8 [01:50<?, ?it/s]
Exception in thread Thread-133:
Traceback (most recent call last):
  File "c:\Users\HP\miniconda3\envs\langchain_env\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\HP\miniconda3\envs\langchain_env\lib\site-packages\ragas\executor.py", line 95, in run
    results = self.loop.run_until_complete(self._aresults())
  File "c:\Users\HP\miniconda3\envs\langchain_env\lib\asyncio\base_events.py", line 649, in run_until_complete
    return future.result()
  File "c:\Users\HP\miniconda3\envs\langchain_env\lib\site-packages\ragas\executor.py", line 83, in _aresults
    raise e
  File "c:\Users\HP\miniconda3\envs\langchain_env\lib\site-packages\ragas\executor.py", line 78, in _aresults
    r = await future
  File "c:\Users\HP\miniconda3\envs\langchain_env\lib\asyncio\tasks.py", line 571, in _wait_for_one
    return f.result()  # May raise f.exception().
  File "c:\Users\HP\miniconda3\envs\langchain_env\lib\site-pac

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.